In [13]:
import numpy as np
import pandas as pd

# read dataset with basic data
# Players with competitive matchs in 2023
df_players = pd.read_csv("data\players.csv", index_col=[0])

# List of champs with competetive stats
df_champs = pd.read_csv("data\champs.csv", index_col=[0])

# List of matchs in 2023
df_matchs = pd.read_csv("data\matchs_2023.csv", index_col=[0])
# Set only the initial players
df_matchs = df_matchs.iloc[:,:-10]

# List the teams and stats
df_teams = pd.read_csv("data/teams.csv", index_col=[0])

In [14]:
# Get the region of each match
def get_region(row, df_teams):
    return df_teams[df_teams["Name"] == row["team_red"]]["Region"].values[0]

df_matchs["region"] = df_matchs.apply(lambda x: get_region(x, df_teams), axis=1)
# df_matchs[df_matchs["region"] == "BR"]

In [15]:
def get_team_idx(team, df_teams):
    return df_teams.index[df_teams["Name"] == team][0]

In [16]:
# Replace the team name for ther id
df_matchs[df_matchs.filter(regex='^team.*').columns.values] = df_matchs.filter(regex='^team.*').applymap(lambda x: get_team_idx(x, df_teams))

In [17]:
#GT, idx, KDA
def get_champ_stats(row, df_champs, stats=["GT","KDA", "Champion"]):
    return df_champs[stats].reset_index()

def get_champ_id(champ, df_champs):
    try:
        id_champ = df_champs.index[df_champs["Champion"] == champ].tolist()[0]
    except:
        id_champ = 999
    return id_champ


In [18]:
# Replace the champ name for ther id
df_matchs[df_matchs.filter(regex='^ban.*').columns.values] = df_matchs.filter(regex='^ban.*').applymap(lambda x: get_champ_id(x, df_champs))

In [19]:
# Replace the champ name for ther id
df_matchs[df_matchs.filter(regex='^pick.*').columns.values] = df_matchs.filter(regex='^pick.*').applymap(lambda x: get_champ_id(x, df_champs))

In [20]:
def get_player_id(player, df_players):
    # print(champ)
    try:
        id_player = df_players.index[df_players["Player"] == player].tolist()[0]
    except:
        id_player = 999
        
    return id_player

In [21]:
# Define the player 
df_matchs[df_matchs.filter(regex='^player.*').columns.values] = df_matchs.filter(regex='^player.*').applymap(lambda x: get_player_id(x, df_players))

In [22]:
def get_match_result(row):
    return [1 if row == "WIN" else 0][0]

# Get the result of each team (0 LOSS, 1 WIN)
df_matchs[df_matchs.filter(regex='^result.*').columns.values] = df_matchs.filter(regex='^result.*').applymap(lambda x: get_match_result(x))

In [23]:
# Train data (converted)
df_matchs = df_matchs.replace("-", 999)
df_matchs

,team_red,result_red,ban_0_red,ban_1_red,ban_2_red,ban_3_red,ban_4_red,pick_0_red,pick_1_red,pick_2_red,...,csM_3_blue,dpM_3_blue,wpM_3_blue,player_4_blue,kda_4_blue,csM_4_blue,dpM_4_blue,wpM_4_blue,match_duration,region
0,20,1,107,147,105,92,999,51,141,1,...,10.5,474,0.30,652,2.00,0.8,248,1.18,00:33:11,FR
0,225,1,157,105,154,1,31,101,75,11,...,10.2,428,999,545,2.08,0.8,246,999,00:30:36,CN
0,90,0,70,136,122,65,143,51,141,2,...,10.3,579,999,433,8.50,0.6,245,999,00:29:39,CN
0,225,1,157,105,154,1,31,101,75,11,...,10.2,428,999,545,2.08,0.8,246,999,00:30:36,CN
0,126,1,154,70,9,157,116,18,142,105,...,8.1,581,999,944,1.27,0.5,226,999,00:46:02,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,147,0,62,11,53,147,38,31,141,122,...,9.9,741,0.31,1151,18.00,1.1,123,1.38,00:39:11,FR
0,207,0,75,105,47,51,2,36,27,11,...,8.4,352,0.34,997,13.00,1.3,115,1.51,00:29:13,FR
0,213,1,105,27,70,51,82,101,75,48,...,8.0,321,0.89,1198,0.40,1.1,74,1.17,00:28:09,FR
0,243,1,70,107,136,41,154,51,27,48,...,6.7,549,0.32,786,3.67,0.9,454,1.24,00:47:35,FR


In [24]:
df_matchs.to_csv("data/match_train_2023.csv")